# **Default Setting**

* python: 3.6.9
* tensorflow: 2.5.0-dev20201208
* cpu: AMD Ryzen 5 5600X 6-Core Processor
* gpu: GeForce RTX 3070 (8G)
* memory: 32G
* platform: linux 20.04 lts

In [61]:
%env NOTEBOOKNAME try1
%env LOCAL_DATA_PATH data
%env SUBMISSION_PATH submission

env: NOTEBOOKNAME=try1
env: LOCAL_DATA_PATH=data
env: SUBMISSION_PATH=submission


In [2]:
import tensorflow as tf
import tensorflow_addons as tfa

import glob
import os
import platform

import numpy as np
import pandas as pd

from collections import OrderedDict
from sklearn.model_selection import train_test_split

print(f"tf.__version__: {tf.__version__}")
print(f"tfa.__version__: {tfa.__version__}")

/home/myunggyo/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201208). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


tf.__version__: 2.5.0-dev20201208
tfa.__version__: 0.11.2


In [3]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
!python -V

Python 3.6.9 :: Anaconda, Inc.


In [5]:
# !cat /proc/cpuinfo

In [6]:
!nvidia-smi

Thu Dec 10 13:09:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 455.32.00    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     On   | 00000000:0A:00.0  On |                  N/A |
|  0%   28C    P8    23W / 220W |    310MiB /  7979MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# !df -h

In [8]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           31Gi       2.0Gi        27Gi        70Mi       1.8Gi        28Gi
Swap:         2.0Gi          0B       2.0Gi


In [9]:
platform.platform()

'Linux-5.4.0-56-generic-x86_64-with-debian-bullseye-sid'

# **Define Arguments**

In [10]:
args = OrderedDict({
    # Basic
    "SEED": 42,
    "TEST_SIZE": 0.2,
    
    # Dataframe
    "NUM_TEST_CSV": 81,
    
    # Dataset Pipeline
    "WINDOW_INP": 7 * 24 * 2, # 336, 7 days
    "WINDOW_TAR": 2 * 24 * 2, # 96, 2 days
    "WINDOW_SHIFT": 1 * 24 * 2, # 1 day
    "WINDOW_STRIDE": 1,
    "WINDOW_DR": True, # Drop remainders
    "AUTO": tf.data.experimental.AUTOTUNE,
    
    "GLOBAL_BATCH_SIZE": 128,
    
})

args

OrderedDict([('SEED', 42),
             ('TEST_SIZE', 0.2),
             ('NUM_TEST_CSV', 81),
             ('WINDOW_INP', 336),
             ('WINDOW_TAR', 96),
             ('WINDOW_SHIFT', 48),
             ('WINDOW_STRIDE', 1),
             ('WINDOW_DR', True),
             ('AUTO', -1),
             ('GLOBAL_BATCH_SIZE', 128)])

# **Load Datasets**

## **Train / Validation**

In [11]:
df = pd.read_csv(os.path.join(os.environ["LOCAL_DATA_PATH"], "train", "train.csv"))
df.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0


In [12]:
df.describe()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
count,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000
mean,547.000000,11.500000,15.000000,64.344121,234.792371,2.456033,56.793102,9.279928,17.790630
std,316.102148,6.922252,15.000143,103.897125,349.684583,1.426874,22.052875,10.179741,25.759955
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.590000,-19.000000,0.000000
25%,273.000000,5.750000,0.000000,0.000000,0.000000,1.400000,39.697500,1.000000,0.000000
50%,547.000000,11.500000,15.000000,0.000000,0.000000,2.200000,57.600000,9.000000,0.000000
75%,821.000000,17.250000,30.000000,87.000000,469.000000,3.200000,72.770000,17.000000,32.089890
max,1094.000000,23.000000,30.000000,528.000000,1059.000000,12.000000,100.000000,35.000000,99.913939


In [13]:
tr_df, vl_df = train_test_split(
    df, 
    test_size = args["TEST_SIZE"], 
    random_state = args["SEED"])

tr_df.shape, vl_df.shape

((42048, 9), (10512, 9))

In [14]:
def preprocessing(df, is_training = True):
    # Drop timestampes.
    try:
        df = df.drop(["Day", "Hour", "Minute"], axis = 1)
    except:
        pass
    
    # Normalize.
    for column in df.columns:
        if is_training:
            args[f"{column}_MAX"] = df[column].max()
            args[f"{column}_MIN"] = df[column].min()
        
        df[column] = (df[column] - args[f"{column}_MIN"]) / (args[f"{column}_MAX"] - args[f"{column}_MIN"])
        
    return df

In [15]:
tr_df = preprocessing(tr_df)
vl_df = preprocessing(vl_df, is_training = False)

In [16]:
tr_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,42048.000000,42048.000000,42048.000000,42048.000000,42048.000000,42048.000000
mean,0.121155,0.221624,0.204474,0.532615,0.523211,0.177645
std,0.195675,0.330325,0.118662,0.238854,0.188508,0.257436
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.116667,0.347040,0.370370,0.000000
50%,0.000000,0.000000,0.183333,0.541392,0.518519,0.000000
75%,0.164773,0.443815,0.266667,0.706309,0.666667,0.320789
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
vl_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,10512.000000,10512.000000,10512.000000,10512.000000,10512.000000,10512.000000
mean,0.124701,0.222062,0.205452,0.531757,0.525669,0.181208
std,0.201099,0.329730,0.119881,0.237798,0.188532,0.261486
min,0.000000,0.000000,0.008333,0.004437,0.037037,0.000000
25%,0.000000,0.000000,0.116667,0.348231,0.370370,0.000000
50%,0.000000,0.000000,0.183333,0.540634,0.518519,0.000000
75%,0.168561,0.438149,0.266667,0.700249,0.666667,0.324526
max,0.990530,0.998111,0.983333,1.000000,1.000000,1.001672


## **Test**

In [18]:
ts_filenames = [os.path.join(os.environ["LOCAL_DATA_PATH"], "test", f"{i}.csv") for i in range(args["NUM_TEST_CSV"])]
ts_dfs = [preprocessing(pd.read_csv(ts_filename), is_training = False) for ts_filename in ts_filenames]
ts_df = pd.concat(ts_dfs)

In [19]:
ts_df.head()

,DHI,DNI,WS,RH,T,TARGET
0,0.0,0.0,0.225000,0.290337,0.351852,0.0
1,0.0,0.0,0.225000,0.287631,0.353704,0.0
2,0.0,0.0,0.225000,0.288280,0.355556,0.0
3,0.0,0.0,0.225000,0.285683,0.357407,0.0
4,0.0,0.0,0.233333,0.285467,0.359259,0.0


In [20]:
ts_df.describe()

,DHI,DNI,WS,RH,T,TARGET
count,27216.000000,27216.000000,27216.000000,27216.000000,27216.000000,27216.000000
mean,0.111880,0.237133,0.189333,0.419916,0.532339,0.182813
std,0.173494,0.338179,0.105967,0.207518,0.209003,0.261818
min,0.000000,0.000000,0.008333,-0.029001,0.061111,0.000000
25%,0.000000,0.000000,0.108333,0.254166,0.359259,0.000000
50%,0.011364,0.000000,0.166667,0.424629,0.512963,0.005645
75%,0.157197,0.477809,0.241667,0.576669,0.690741,0.326398
max,0.960227,1.018886,0.916667,0.919597,1.031481,0.997030


# **Make Dataset Pipelines**

In [21]:
@tf.function
def _flat_fn_tr(x):
    return x.batch(args["WINDOW_INP"] + args["WINDOW_TAR"])


@tf.function
def _flat_fn_ts(x):
    return x.batch(args["WINDOW_INP"])


@tf.function
def _split_window(features):
    return tf.split(features, [args["WINDOW_INP"], args["WINDOW_TAR"]], axis = 0)

In [22]:
tr_tensor = tf.constant(tr_df, dtype = tf.float32)
vl_tensor = tf.constant(vl_df, dtype = tf.float32)
ts_tensor = tf.constant(ts_df, dtype = tf.float32)

tr_dataset = tf.data.Dataset.from_tensor_slices(tr_tensor
                    ).window(args["WINDOW_INP"] + args["WINDOW_TAR"], args["WINDOW_SHIFT"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_tr
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
                    ).cache(
                    ).prefetch(args["AUTO"])

vl_dataset = tf.data.Dataset.from_tensor_slices(vl_tensor
                    ).window(args["WINDOW_INP"] + args["WINDOW_TAR"], args["WINDOW_SHIFT"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_tr
                    ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["GLOBAL_BATCH_SIZE"]
                    ).cache(
                    ).prefetch(args["AUTO"])

ts_dataset = tf.data.Dataset.from_tensor_slices(ts_tensor
                    ).window(args["WINDOW_INP"], args["WINDOW_INP"], args["WINDOW_STRIDE"], args["WINDOW_DR"]).flat_map(_flat_fn_ts
#                     ).map(_split_window, num_parallel_calls = args["AUTO"]
                    ).batch(args["NUM_TEST_CSV"] # Not args["GLOBAL_BATCH_SIZE"]
#                     ).cache(
                    ).prefetch(args["AUTO"])

# Print the shapes
print(f"tr_dataset: {tr_dataset.element_spec}")
print(f"vl_dataset: {vl_dataset.element_spec}")
print(f"ts_dataset: {ts_dataset.element_spec}")

tr_dataset: (TensorSpec(shape=(None, 336, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 6), dtype=tf.float32, name=None))
vl_dataset: (TensorSpec(shape=(None, 336, 6), dtype=tf.float32, name=None), TensorSpec(shape=(None, 96, 6), dtype=tf.float32, name=None))
ts_dataset: TensorSpec(shape=(None, None, 6), dtype=tf.float32, name=None)


## **Take Samples**

In [189]:
for element in tr_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 336, 6) (128, 96, 6)


In [190]:
for element in vl_dataset.take(1):
    foo, bar = element
    print(foo.shape, bar.shape)

(128, 336, 6) (128, 96, 6)


In [191]:
for element in ts_dataset.take(1):
    foo = element
    print(foo.shape)

(81, 336, 6)


# **Create Network**

In [41]:
OUT_STEPS = int(args["WINDOW_TAR"] / 48)

class MultiStepLastBaseline(tf.keras.Model):
    def call(self, inputs):
        return [tf.tile(inputs[:, -48:, :], [1, OUT_STEPS, 1])] * 9

# **Compile and Fit**

   * Repeat the last 24 hours.

In [42]:
last_baseline = MultiStepLastBaseline()
last_baseline.compile(
    loss = [tfa.losses.PinballLoss(tau = i) for i in np.arange(0.1, 1, 0.1)])

In [43]:
last_baseline.evaluate(tr_dataset)

7/7 [==============================] - 0s 2ms/step - loss: 1.0232 - output_1_loss: 0.1137 - output_2_loss: 0.1137 - output_3_loss: 0.1137 - output_4_loss: 0.1137 - output_5_loss: 0.1137 - output_6_loss: 0.1137 - output_7_loss: 0.1137 - output_8_loss: 0.1137 - output_9_loss: 0.1137


[1.022362232208252,
 0.11359583586454391,
 0.11359582841396332,
 0.11359582841396332,
 0.11359581351280212,
 0.11359579861164093,
 0.11359578371047974,
 0.11359577625989914,
 0.11359576135873795,
 0.11359576135873795]

In [44]:
last_baseline.evaluate(vl_dataset)

2/2 [==============================] - 0s 6ms/step - loss: 1.0355 - output_1_loss: 0.1150 - output_2_loss: 0.1150 - output_3_loss: 0.1150 - output_4_loss: 0.1150 - output_5_loss: 0.1151 - output_6_loss: 0.1151 - output_7_loss: 0.1151 - output_8_loss: 0.1151 - output_9_loss: 0.1151


[1.0354653596878052,
 0.11503617465496063,
 0.11504006385803223,
 0.11504393815994263,
 0.11504781991243362,
 0.11505170166492462,
 0.11505559086799622,
 0.11505946516990662,
 0.11506334692239761,
 0.11506723612546921]

# **Predict**

In [55]:
pred = np.array(last_baseline.predict(ts_dataset))[..., -1] # (9, 81, 96)
pred = np.transpose(pred, axes = (1, 2, 0)) # (81, 96, 9)
pred = np.reshape(pred, (-1, pred.shape[-1]))
pred.shape

(7776, 9)

In [46]:
sample_submission = pd.read_csv(os.path.join(os.environ["LOCAL_DATA_PATH"], "sample_submission.csv"), index_col = "id")
sample_submission.head()

,q_0.1,q_0.2,q_0.3,q_0.4,q_0.5,q_0.6,q_0.7,q_0.8,q_0.9
id,,,,,,,,,
0.csv_Day7_0h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_0h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_1h30m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.csv_Day7_2h00m,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
sample_submission[:] = pred

In [62]:
sample_submission.to_csv(os.path.join(os.environ["SUBMISSION_PATH"], f"{os.environ['NOTEBOOKNAME']}_submission.csv"))